<a href="https://colab.research.google.com/github/jyp1111/TIL/blob/master/nlp/textrank_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.5MB/s 
     |████████████████████████████████| 92kB 7.6MB/s 
     |████████████████████████████████| 3.8MB 37.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
import numpy as np
from itertools import permutations
from konlpy.tag import Okt
import requests
from bs4 import BeautifulSoup

In [ ]:
class textrank:    
    def __init__(self,token_ls):
        self.token_ls=token_ls
        self.word_ls=list(set(self.token_ls))

    def connected_matrix(self,windows_size):
        n=len(self.word_ls)
        Mat=np.zeros((n,n))

        for v in permutations(range(len(self.token_ls)),2): 
            if abs(v[0]-v[1])<windows_size:
                Mat[self.word_ls.index(self.token_ls[v[0]])][self.word_ls.index(self.token_ls[v[1]])]=1

        result_Mat=np.array([v/sum(v) for v in Mat])

        return result_Mat

    def score(self,d,windows_size,num_epoch,threshold):
        score=np.ones(len(self.word_ls))
 
        for i in range(num_epoch):
            new_score=(1-d)+np.dot(self.connected_matrix(windows_size).T,d*score)
            if (np.abs(score-new_score)).mean()<threshold:
                break
            score=new_score

        return new_score

In [ ]:
L=["딸기","바나나","사과","딸기","파인애플"]
textrank(L).score(d=0.85,windows_size=2,num_epoch=20,threshold=0.001)

array([1.46623101, 0.98390456, 0.98390456, 0.56595988])

In [ ]:
def make_word_ls(url):
    res=requests.get(url)
    soup=BeautifulSoup(res.text,"html.parser")
    result=soup.select_one("#articleBodyContents")
    result_text=result.text.replace("\n","").replace("\t","").replace("function _flash_removeCallback() {}","").replace("// flash 오류를 우회하기 위한 함수 추가","")
    for x in result.select("a"):
        result_text=result_text.replace(x.text,"")
    result_text=result_text.replace("▶","").replace("무단 전재 및 재배포 금지","")
    
    okt= Okt()
    okt_tokens = okt.morphs(result_text)
    oktTag=sum([okt.pos(token) for token in okt_tokens],[])

    stopPos=["Punctuation","Josa","Foreign","Adverb","URL","Verb","Conjunction"]

    keyword_ls = [tag[0] for tag in oktTag if (tag[1] not in stopPos) and (len(tag[0])>1)]

    word_ls = [tag[0] for tag in oktTag if tag[1] not in stopPos]

    remove_index_ls=sorted(list(set([i for i,tag in enumerate(oktTag) if tag[1] in stopPos]+[0,len(oktTag)])))
    words_ls=[[tag[0] for tag in oktTag[remove_index_ls[i]:remove_index_ls[i+1]] if tag[1] not in stopPos] 
              for i in range(len(remove_index_ls)-1) if remove_index_ls[i+1]-remove_index_ls[i]>1]

    return keyword_ls,word_ls,words_ls

def keyword_recommand(url,num,d,windows_size,num_epoch,threshold):
    keyword_ls,_,_=make_word_ls(url)

    score=textrank(keyword_ls).score(d,windows_size,num_epoch,threshold)
    sorted_word_ls=sorted(textrank(keyword_ls).word_ls, key=lambda word: score[textrank(keyword_ls).word_ls.index(word)],reverse=True)

    return sorted_word_ls[:num]

def words_recommand(url,num,d,windows_size,num_epoch,threshold):
    _,word_ls,words_ls=make_word_ls(url)

    score=textrank(word_ls).score(d,windows_size,num_epoch,threshold)
    score_ls=[sum([score[textrank(word_ls).word_ls.index(word)] for word in words]) for words in words_ls]
    sorted_words_ls=sorted(words_ls, key=lambda words: score_ls[words_ls.index(words)],reverse=True)

    return sorted_words_ls[:num]

In [ ]:
url="https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=103&oid=025&aid=0003018110"
keyword_recommand(url=url,num=5,d=0.85,windows_size=3,num_epoch=20,threshold=0.001)

['해신', '재료', '불도장', '음식', '이름']

In [ ]:
words_recommand(url=url,num=10,d=0.85,windows_size=3,num_epoch=20,threshold=0.001)

[['12',
  '종류',
  '의',
  '부',
  '재료',
  '를',
  '포함',
  '해',
  '총',
  '30',
  '여',
  '종',
  '의',
  '식',
  '재료'],
 ['각', '재료', '의', '맛', '과', '영양', '이', '충분히', '우러날', '수', '있다'],
 ['이', '곳', '의', '김성', '완', '조리', '장', '은', '2005년', '여름', '드라마'],
 ['그', '중', '가장', '대중', '적', '인', '삼계탕', '은', '열량', '이', '풍부한', '고기'],
 ['수', '있는', '식', '재료', '중', '영양', '이', '가장', '풍부한', '낙지'],
 ['대부분', '의', '전통', '보양식', '이', '식', '재료'],
 ['예', '부터', '복날', '이면', '그', '해', '의', '더위', '를', '물리'],
 ['송이버섯', '등', '의', '식', '재료', '를', '메인'],
 ['은', '제철', '식', '재료', '를', '이용', '해', '조선시대', '사대부', '요리'],
 ['위해', '7~8월', '두', '달', '동안', '투숙', '객', '이', '해신', '탕']]